# Bottlenecks

> High-performance computing is computing at the limit

CPU

Memory

I/O

## CPU

### On JURECA

Each core runs at 2.5 GHz
- 256 bit wide vector unit (4 double precision numbers)
- 2 Fused multiply-add operations per cycle
- Peak performance is 2.5 * 4 *2 * 2 GFlop/s = 40 GFlop/s

There are 12 cores per socket
- Peak performance of a socket is 12 * 40 GFlop/s = 480 GFlop/s

There are two sockets per node
- Peak performance of a node (without GPUs) 960 GFlop/s

This is the limit most people think of first, but it's often not the crucial one. Each core on JURECA can perform 40 GFlop/s if the code is completely *vectorized* and performs a *multiply and an add operation* at *each step*. If your code doesn't fulfill those requirements its peak performance will be less.

Actually the peak performance is a little less because the AVX vector units run at a slightly lower frequency than the processor

## Memory bandwidth

The memory bandwidth of JURECA is about 60 GB/s per socket (12 cores). Let's look at a simple operation:

c = c + a * b

I assume that we are dealing with double precision numbers (8 bytes) then I have to read 3 * 8 bytes = 24 bytes and write 8 bytes. This is a multiply add operation, so each core can do 20 billion of those per second, but it only receives 60 GB/s / 24 bytes/op = 2.5Gop/s. This operation is clearly memory bound, if we have to get all the data from main memory.

This is quite common. Let's look at a matrix multiplication $C=AB$. To calculate the element i, j of the result matrix C, we multiply row i of A with column j of B and sum the results. This is the scalar or dot product of row i of A and column j of B. In code this looks like this:

In [1]:
import numpy as np
def dot(a, b):
    """Multiply the matrix a with the matrix b.
    
    Parameters
    ----------
    a: ndarray
        left matrix
    b: ndarray
        right matrix
    
    Return
    ------
    c: ndarray
        result matrix
    """
    c = np.zeros((a.shape[0], b.shape[1]))   
    for i in range(a.shape[0]):             
        for j in range(b.shape[1]):         
            for k in range(a.shape[1]):     
                 c[i, j] += a[i, k] * b[k, j]
    return c                                
 

Let's take two small matrices A and B and see how long the above function takes.

In [2]:
n = 256
a = np.random.random((n, n))
b = np.random.random((n, n))

In [3]:
%timeit dot(a,b)

1 loop, best of 3: 9.57 s per loop


A matrix multiplication of two n by n matrices performs $2n^3$ operations. The dot function achieves

In [4]:
print("%.3f GFLOP/s" % (2e-9 * n**3 / 9.57)) # Replace the last number with the time measured by %timeit

0.004 GFLOP/s


Wow, that's bad. Let's see if we can make this faster.

### Numba

In [5]:
from numba import jit
jdot = jit(dot)

In [6]:
%timeit jdot(a, b)

The slowest run took 18.57 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 22.9 ms per loop


In [8]:
print("%.3f GFLOP/s" % (2e-9 * n**3 / 0.0229)) # Replace the last number with the time measured by %timeit converted to seconds.

1.465 GFLOP/s


### Access order and cache lines

From our estimate above, we should be able to get at least twice this, but that's assuming we can achieve the maximum memory bandwidth. 

A numpy ndarray uses C-order (row-order) for storage. This means that the last index is continuous in memory and a change in any other index results in a jump from one memory location to another. The order of the loops therefore means that for both c and b, we don't get the maximum bandwidth, because we jump around and only use one element of the cache line. 

A datum is not loaded by itself. Instead everytime, a datum is needed that is not available in cache, a cache line containing the datum is loaded. On JURECA the cache line is 64 bytes wide. 

We can improve the performance by exchanging the loops:

In [9]:
import numpy as np
from numba import jit

@jit
def dot2(a, b):
    """Multiply the matrix a with the matrix b.
    
    Parameters
    ----------
    a: ndarray
        left matrix
    b: ndarray
        right matrix
    
    Return
    ------
    c: ndarray
        result matrix
    """
    c = np.zeros((a.shape[0], b.shape[1]))   
    for i in range(a.shape[0]):             
        for k in range(a.shape[1]):     
            for j in range(b.shape[1]):         
                 c[i, j] += a[i, k] * b[k, j]
    return c                                
 

Now, elements in c and b are accessed in the proper order and a[i, k] is constant for the loop. This changes our estimate, because, now we read 16 bytes/op. This gives us a maximum of 60 GB/s / 16 bytes/op = 3.75 Gop/s.

In [10]:
%timeit dot2(a, b)

The slowest run took 38.71 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 6.31 ms per loop


In [11]:
print(2e-9 * n**3 / 0.00631, "GFLOP/s") # Replace the last number with the time measured by %timeit converted to seconds.

5.317659587955626 GFLOP/s


This is better than what we would expect from the bandwidth estimate, probably due to caching. One way to test this is to make the matrix larger, so that it doesn't fit in cache anymore.

In [12]:
n = 4096
a = np.random.random((n,n))
b = np.random.random((n,n))

In [13]:
%timeit dot2(a,b)

1 loop, best of 3: 42 s per loop


In [14]:
print(2e-9 * n**3 / 44.5, "GFLOP/s") # Replace the last number with the time measured by %timeit converted to seconds.

3.088515808359551 GFLOP/s


This is quite close to my estimate and shows that I have to increase the number of operations per byte loaded from main memory. Improving vectorization or using multiple threads wouldn't help.

To improve cache utilization, we have to change the algorithm. One way to improve the performance of the matrix multiplication is blocking (aka tiling). This is done, e.g., in OpenBLAS or Intel's Math Kernel Library (MKL).

### Numpy

Let's see how long numpy takes for this:

In [ ]:
%timeit np.dot(a, b)

In [ ]:
print(2e-9 * n**3 / 1.53, "GFLOP/s") # Replace the last number with the time measured by %timeit converted to seconds

The numpy version we use here, uses a fast math library. That's what you want.

## I/O

### IBM GPFS
$\mathcal{O}(100)$ GB/s read/write speed

Each node connected to file system with 10 GBit/s or 1.25 GB/s.

Our GPFS achieves read/write bandwidths that are very similar to the main memory bandwidth, but not for a single node. Each node is connected to the GPFS with a 10 GBit/s connection. In other words, we can read/write about 1.25 GB/s. If we had to load the data in the previous calculation from disk, we could only achieve 1.25 GB/s / 24 bytes/op = 52 Mop/s. The main memory bandwidth or the peak performance of the CPU doesn't matter in this case.